# 部分宏观指标数据转化为月度数据

In [12]:
import pandas as pd
import numpy as np
import datetime
import os

In [13]:
data_origin_file = os.listdir('部分宏观指标数据库')
data_origin = {}
for file in data_origin_file:
    try:
        data_temp = pd.read_excel('部分宏观指标数据库/'+file)
        frequency = data_temp.iloc[0, 1]
        data_temp = data_temp.iloc[1:data_temp.shape[0]-2]
        data_temp['date'] = data_temp['指标名称']
        del data_temp['指标名称']
    except:
        continue
    else:
        data_origin[file[:-5]] = {'data': data_temp, 'frequency': frequency}

In [14]:
file_name = list(data_origin.keys())

In [15]:
# 转化为月度数据
data_monthly = {} # 存放月度数据的字典
for file in file_name:
    if data_origin[file]['frequency'] != '年':
        data = data_origin[file]['data'].copy()
        if data_origin[file]['frequency'] != '月':
            data['date'] = data['date'].apply(lambda x: datetime.datetime.strptime(x[:-3], '%Y-%m'))
            data.iloc[:, 0] = data.iloc[:, 0].astype(float)
            data_monthly[file] = data.groupby('date').mean()
        else:
            data['date'] = data['date'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m'))
            data = data.set_index('date')
            data_monthly[file] = data

In [16]:
# 计算月度环比
data_mom = {} # 存放月度环比数据的字典
for file in data_monthly.keys():
    data_mom[file] = data_monthly[file].pct_change(periods=1)

In [17]:
# 计算月度同比
data_yoy = {} # 存放月度同比数据的字典
for file in data_monthly.keys():
    data = data_monthly[file].copy()
    data_ = data.copy()
    column_name = data.columns[0]
    for i in range(len(data.index)):
        last_year = datetime.datetime(data.index[i].year-1, data.index[i].month, data.index[i].day)
        if last_year in data.index:
            data_.loc[data.index[i], column_name] = data.loc[data.index[i], column_name] / data.loc[last_year, column_name] - 1
        else:
            data_.loc[data.index[i], column_name] = np.nan
    data_yoy[file] = data_

In [18]:
# 筛选日期
for file in data_monthly.keys():
    data_monthly[file] = data_monthly[file].loc[data_monthly[file].index>='2000-01-01']
    data_mom[file] = data_mom[file].loc[data_mom[file].index>='2000-01-01']
    data_yoy[file] = data_yoy[file].loc[data_yoy[file].index>='2000-01-01']

In [19]:
# 存储数据
# 存储月度数据
for file in data_monthly.keys():
    data_monthly[file].to_excel('部分宏观指标数据库-转化月度数据/'+file+'-月度.xlsx')
    data_mom[file].to_excel('部分宏观指标数据库-转化月度数据/'+file+'-月度环比.xlsx')
    data_yoy[file].to_excel('部分宏观指标数据库-转化月度数据/'+file+'-月度同比.xlsx')